To work with partitions, we will use `AbstractAbgebra` module. Define a shortcut `AbstractPartition = AbstractAlgebra.Generic.Partition{Int64}`

In [1]:
using AbstractAlgebra
import Combinatorics
AbstractPartition = AbstractAlgebra.Generic.Partition{Int64}
const empty_partition = Partition(Int[])

The goal is to compute the translation matrix

$$
J_\lambda^{(\alpha)} = \sum_{\mu \vdash n}\theta_\lambda(\mu) p_\mu,
$$

where $J_\lambda^{(\alpha)}$ are Jack polynomials, and $p_\mu$ are the power sum symmetric polynomials.

The first approach is based on the formula from the paper `Lassalle, Michel. "Jack polynomials and free cumulants." Advances in Mathematics 222.6 (2009): 2227-2269.` First, define

$$
c_i(\lambda) = \frac{1}{\alpha\lambda_i + l(\lambda) - i + 2} \prod_{\substack{j=1\\j\ne i}} \frac{\alpha(\lambda_i - \lambda_j) + j - i + 1}{\alpha(\lambda_i - \lambda_j) + j - i}
$$

Next, the reccurent relation on $\theta_\lambda(\mu)$ is the following:

$$
\begin{align*}
\theta_\lambda(\mu_{\downarrow(1)}) &= \sum_{i = 1}^{l(\lambda) + 1}c_i(\lambda)\theta_{\lambda^{(i)}}(\mu),\\
\sum_{r \ge 1}(m_r(\mu) + 1)\theta_\lambda(\mu_{\downarrow(r + 1)} &= \sum_{i = 1}^{l(\lambda)+ 1}c_i(\lambda)(\lambda_i - (i - 1) / \alpha)\theta_{\lambda^{(i)}}(\mu)
\end{align*}
$$

This system totally determines all $\theta$ (see Theorem 3.1). Also, Theorem 3.1 provides an efficient reccurence relation on $\theta$, which I implement next.

The struct PartitionLambda is defined for each partition $\lambda$. It contains precomputed values of the polynomials 
$$
\begin{align*}
&\tilde{c}_i(\lambda) \colon = c_i(\lambda) \cdot \mathrm{mult}(\lambda), \\
&\mathrm{mult}(\lambda) := \prod_{i = 1}^{l(\lambda) + 1}(\alpha\lambda_i + l(\lambda) - i + 2) \cdot \prod_{\substack{i, j=1\\j\ne i}}^{l(\lambda) + 1}(\alpha(\lambda_i - \lambda_j) + j - i)
\end{align*}
$$ 

and the struct PartitionRho is defined for each partition $\rho$ and contains precomputed values $m_r(\rho)$ and the vector of partitions $\rho|_{\downarrow(r)}$, $1 \le r \le |\lambda|$ (in the code, it is denote by `downᵣ`)

In [2]:
struct PartitionLambda{GenericPolynomial}
    partition::AbstractPartition
    c::Vector{GenericPolynomial}
    mult::GenericPolynomial
end

function PartitionLambda(λ::AbstractPartition, α::GenericPolynomial) where GenericPolynomial
    c = fill(one(α), length(λ) + 1)
    for i = 1:(length(λ) + 1)
        for p = 1:(length(λ) + 1)
            λₚ = (p <= length(λ) ? λ.part[p] : 0)
            if p != i
                c[i] *= (α * λₚ + length(λ) - p + 2)
            end            
            for j = 1:(length(λ) + 1)
                if j == p
                    continue
                end
                λⱼ = (j <= length(λ) ? λ.part[j] : 0)
                δ = (i == p ? 1 : 0)
                c[i] *= α * (λₚ - λⱼ) + j - p + δ
            end
        end
    end

    mult = one(α)
    for p = 1:(length(λ) + 1)
        λₚ = (p <= length(λ) ? λ.part[p] : 0)
        mult *= (α * λₚ + length(λ) - p + 2)
        for j = 1:(length(λ) + 1)
            if j == p
                continue
            end
            λⱼ = (j <= length(λ) ? λ.part[j] : 0)
            mult *= α * (λₚ - λⱼ) + j - p
        end
    end
    PartitionLambda{GenericPolynomial}(λ, c, mult)
end


struct PartitionRho
    partition::AbstractPartition
    mᵣ::Vector{Int}
    downᵣ::Vector{AbstractPartition}
end

function PartitionRho(ρ::AbstractPartition)
    mᵣ = fill(0, ρ.n)
    downᵣ = [empty_partition for ~ = 1:ρ.n]
    for i = length(ρ):-1:1
        mᵣ[ρ.part[i]] += 1
        if !(i == length(ρ) || ρ[i] > ρ[i + 1])
            continue
        end
        if ρ.part[i] == 1
            downᵣ[1] = Partition(ρ.part[begin:end-1])
        else
            downᵣ[ρ.part[i]] = deepcopy(ρ)
            downᵣ[ρ.part[i]].part[i] -= 1
        end
    end
    return PartitionRho(ρ, mᵣ, downᵣ)
end

PartitionRho

In [3]:
using ProgressMeter

function up(λ::AbstractPartition, i::Int)
    if i == 1 || λ.part[i] < λ.part[i - 1]
        λⁱ = deepcopy(λ)
        λⁱ.part[i] += 1
        return λⁱ
    end
    return empty_partition
end    


function generate_θ_matrix(N::Int, α::GenericPolynomial) where GenericPolynomial
    θ = Dict()
    θ[(Partition([1]), Partition([1]))] = one(α)
    
    function get_θ(λ, ρ)
        if λ != Partition(Int[]) && ρ != Partition(Int[])
            return θ[(λ, ρ)]
        end
        return zero(α)
    end  
    
    buffer = one(α) 

    @showprogress for n = 2:N
        partitions = Combinatorics.integer_partitions(n)
        partitions = map(
            x -> Partition(x),
            sort!(partitions, by=(x -> (length(x), x[end])))
        )
        ρs_data = Dict(
            ρ => PartitionRho(ρ) for ρ in partitions
        )
        for λ⁺ in partitions
            if λ⁺.part[end] == 1
                λ = Partition(λ⁺.part[begin:end-1])
            else
                λ = copy(λ⁺.part)
                λ[end] -= 1
                λ = Partition(λ)
            end
            λ_data = PartitionLambda(λ, α)

            for ρ in partitions
                if (λ⁺, ρ) in keys(θ)
                    continue
                end
                ρ_data = ρs_data[ρ]

                if λ⁺.part[end] == 1
                    rhs = get_θ(λ, ρ_data.downᵣ[1]) * λ_data.mult
                    for i = 1:length(λ)
                        addmul!(rhs, -λ_data.c[i], get_θ(up(λ, i), ρ), buffer)
                    end
                    θ[(λ⁺, ρ)] = div(rhs, λ_data.c[length(λ) + 1])
                    continue
                end

                rhs = [zero(α), zero(α)]

                addeq!(rhs[1], get_θ(λ, ρ_data.downᵣ[1]) * λ_data.mult)
                for r = 1:(n - 1)
                    addmul!(
                        rhs[2],
                        r * (ρ_data.mᵣ[r] + 1) * α * get_θ(λ, ρ_data.downᵣ[r + 1]),
                        λ_data.mult, buffer
                    )
                end

                for i = 1:(length(λ) - 1)
                    term = λ_data.c[i] * get_θ(up(λ, i), ρ)
                    addeq!(rhs[1], -term)
                    addmul!(rhs[2], -term, (α * λ.part[i] - (i - 1)), buffer)
                end
                θ[λ⁺, ρ] = div(
                    rhs[1] * length(λ) + rhs[2],
                    (α * λ.part[end] + 1) * λ_data.c[length(λ)]
                )

                λ₁ = deepcopy(λ)
                push!(λ₁.part, 1)
                θ[λ₁, ρ] = div(
                    rhs[1] * (α * λ.part[end] - (length(λ) - 1)) - rhs[2],
                    (α * λ.part[end] + 1) * λ_data.c[length(λ) + 1]
                )
            end
        end
    end
    return θ
end

generate_θ_matrix (generic function with 1 method)

In [4]:
import Nemo
R, α = Nemo.PolynomialRing(Nemo.QQ, "α")
θ = generate_θ_matrix(20, α)


Welcome to Nemo version 0.27.2

Nemo comes with absolutely no warranty whatsoever



Progress: 100%|█████████████████████████████████████████| Time: 0:03:19


Dict{Any, Any} with 995073 entries:
  (8₁7₁, 7₁2₄)           => 75600*α^10 - 719460*α^9 + 19110*α^8 - 306390*α^7 - …
  (6₂3₁1₃, 14₁3₁1₁)      => -518400*α^12 + 6212160*α^11 - 31904352*α^10 + 72347…
  (6₁4₁3₁1₂, 6₁5₁3₁1₁)   => -2880*α^10 + 18336*α^9 - 67080*α^8 + 202352*α^7 - 2…
  (6₁3₁2₄1₃, 6₂4₁2₂)     => 92160*α^11 - 658080*α^10 + 6092640*α^9 - 32120280*α…
  (15₁2₁1₃, 12₁4₁2₂)     => 87178291200*α^14 - 348713164800*α^13 + 326918592000…
  (15₁2₂1₁, 8₁4₂1₄)      => -45729684000*α^12 + 158334976800*α^11 - 80140460400…
  (10₁4₁3₁2₁1₁, 14₁1₆)   => -2177280*α^12 + 146813760*α^11 - 1072519488*α^10 + …
  (5₁4₂3₁2₁1₂, 3₁2₅1₇)   => 20358*α^7 - 304176*α^6 + 1585968*α^5 - 4187268*α^4 …
  (4₃3₂, 10₁2₂1₄)        => 121824*α^9 - 2893968*α^8 + 15127992*α^7 - 36806940*…
  (7₁6₁, 5₂1₃)           => 72576*α^8 + 38880*α^7 + 100080*α^6 - 21600*α^5 + 36…
  (9₁8₁2₁, 10₁4₁1₅)      => -577051776*α^11 + 1355393088*α^10 - 889676784*α^9 +…
  (8₁4₂1₄, 6₁4₁3₁1₇)     => 322560*α^10 - 3414144*α^9 + 23637504*α^8 - 87

The second approach is based on the paper `Haglund, James, and Andrew Timothy Wilson. "Macdonald polynomials and chromatic quasisymmetric functions." arXiv preprint arXiv:1701.05622 (2017).` The formula is the following (see Corollary 4.3.1):

$$
J_{\mu'}^{(\alpha)} = \sum_{H \subset G_\mu^+}(-1)^{|H|} \prod_{\{u, v\} \in H \backslash G_\mu} -\mathrm{hook}_\mu^{(\alpha)}(u)
$$

where $|H|$ is the number of edges in $H$ and $\lambda(H)$ is the partition whose parts equal to the sizes of the connected components of the graph induced by $H$. The definitions of $G_\mu$ and $G_\mu^+$ can be found in the preprint.

First, we implement the function $\lambda(H)$:

In [5]:
function graph_component_size(n::Int, edges::Vector{Pair{Int, Int}})
    color = fill(0, n)
    current_color = 0
    
    graph = Vector{Int}[Int[] for ~ = 1:n]
    for edge in edges
        push!(graph[edge.first], edge.second)
        push!(graph[edge.second], edge.first)
    end
    
    function dfs(u)
        if color[u] != 0
            return
        end
        color[u] = current_color
        for v in graph[u]
            dfs(v)
        end
    end
    
    for u = 1:n
        if color[u] == 0
            current_color += 1
            dfs(u)
        end
    end
    
    partition = fill(0, current_color)
    for u = 1:n
        partition[color[u]] += 1
    end
    return Partition(sort(partition)[end:-1:begin])
end

graph_component_size (generic function with 1 method)

The struct `PartitionGraphData` is defined for the partition $\mu$ and contains the precomputed functions $\mathrm{leg}(u)$, $\mathrm{arm}(u)$, and $\mathrm{down}(u)$, which are defined in `Haglund, James, and Andrew Timothy Wilson. "Macdonald polynomials and chromatic quasisymmetric functions." arXiv preprint arXiv:1701.05622 (2017).` Also, this structure contains the edges of the graphs $G_\mu$ and $G_\mu^+$.

In [6]:
struct PartitionGraphData
    leg::Vector{Int}
    arm::Vector{Int}
    down::Vector{Int}
    G_μ::Vector{Pair{Int, Int}}
    G_μ⁺::Vector{Pair{Int, Int}}
end

function PartitionGraphData(partition::AbstractPartition)
    part = partition.part[end:-1:begin]
    n = partition.n
    l = length(partition)
    filling = fill(0, l, part[end])
    position = fill(Pair(0, 0), n)
    
    u = 1
    for (num_row, len_row) in enumerate(part)
        for i = 1:len_row
            filling[num_row, i] = u
            position[u] = Pair(num_row, i)
            u += 1
        end
    end
    
    leg = fill(0, n)
    arm = fill(0, n)
    down = fill(0, n)
    G_μ = Pair{Int, Int}[]

    for u = 1:n
        row, column = position[u].first, position[u].second
        
        arm[u] = part[row] - column
        for v ∈ filling[row, (column + 1):part[row]]
            push!(G_μ, Pair(u, v))
        end

        if row != l
            for v ∈ filling[row + 1, 1:(column - 1)]
                push!(G_μ, Pair(u, v))
            end
            down[u] = filling[row + 1, column]
        end
        
        for j = (row - 1):-1:1
            if part[j] >= column
                leg[u] += 1
            else
                break
            end
        end
    end
    
    G_μ⁺ = copy(G_μ)
    for u = 1:n
        if down[u] != 0
            push!(G_μ⁺, Pair(u, down[u]))
        end
    end
    return PartitionGraphData(leg, arm, down, G_μ, G_μ⁺)
end

PartitionGraphData

Finally, the function `get_θ_coefs_using_graphs` computes the coefficients $\theta_\mu(\cdot)$ for the given partition $\mu$. The summation formula is used here.

In [7]:
function get_θ_coefs_using_graphs(μ::AbstractPartition, α::GenericPolynomial) where GenericPolynomial
    n = μ.n
    μ = conj(μ)
    data = PartitionGraphData(μ)
    partitions = Combinatorics.integer_partitions(n)
    
    θ = Dict(Partition(λ) => zero(α) for λ ∈ partitions)
    for H in Combinatorics.powerset(data.G_μ⁺)
        λ = graph_component_size(n, H)
        summand = (-1)^length(H)
        for edge in setdiff(H, data.G_μ)
            u = edge.first
            summand *= (-1) * (α * (data.leg[u] + 1) + data.arm[u])
        end
        θ[λ] += summand
    end
    return θ
end

get_θ_coefs_using_graphs (generic function with 1 method)

Now, we can check that the $\theta$ coefficients produced by `generate_θ_matrix` and `get_θ_coefs_using_graphs` are identical. We check it for all partitions $\lambda$ and $\mu$ of the size 7. 

In [8]:
N = 7

θ = generate_θ_matrix(N, α)
partitions = map(Partition, Combinatorics.integer_partitions(N))
for λ ∈ partitions
    θ_graph = get_θ_coefs_using_graphs(λ, α)
    for μ ∈ partitions
        @assert θ_graph[μ] == θ[(λ, μ)]
    end
end
println("We are awesome!")

We are awesome!


We can generate the Jack characters $\omega_\lambda(\mu)$ defined as follows:

$$
p_\mu = \sum_{\lambda \vdash n} \omega_\lambda(\mu) \cdot J^{(\alpha)}_\lambda.
$$

First, we compute the function z(μ) for any partition $\mu$.

In [9]:
function z(λ)
    t = zeros(Int, λ.n)
    for i in λ.part
        t[i] += 1
    end
    result = 1
    for i = 1:(λ.n)
        result *= i^t[i] * factorial(t[i])
    end
    return result
end

z (generic function with 1 method)

We will use the formula:

$$
\omega_\lambda(\mu) = \theta_\lambda(\mu) \cdot z_\mu \cdot \alpha^{l(\mu)} / j_\lambda(\alpha),
$$

where

$$
j_\lambda = \langle J^{(\alpha)}_\lambda, J^{(\alpha)}_\lambda \rangle_\alpha =  \sum_{\mu \vdash n}\theta_\lambda(\mu)^2 \cdot z_\mu \cdot \alpha^{l(\mu)}
$$

In [10]:
function compute_j_λ(λ, θ, α)
    return sum(
        (θ[(λ, μ)] ^2 * z(μ) * α^(length(μ)))
        for μ ∈ map(Partition, Combinatorics.integer_partitions(λ.n))
    )
end 

compute_j_λ (generic function with 1 method)

In [11]:
function generate_ω_matrix(N::Int, α::GenericPolynomial) where GenericPolynomial
    θ = generate_θ_matrix(N, α)
    ω = Dict()
    partitions = map(Partition, Combinatorics.integer_partitions(N))
    for λ ∈ partitions
        j_λ = compute_j_λ(λ, θ, α)
        for μ ∈ partitions
            ω[(λ, μ)] = θ[(λ, μ)] * z(μ) * α^(length(μ)) // j_λ
        end
    end
    return ω
end

generate_ω_matrix (generic function with 1 method)

In [12]:
N = 6

R, α = PolynomialRing(QQ, "α")
S, β = RationalFunctionField(QQ, "α")
ω = generate_ω_matrix(N, α)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Dict{Any, Any} with 121 entries:
  (1₆, 2₃)       => -α^2//(α^5 + 15*α^4 + 85*α^3 + 225*α^2 + 274*α + 120)
  (6₁, 3₂)       => 1//120//(α^5 + 137//60*α^4 + 15//8*α^3 + 17//24*α^2 + 1//8*…
  (1₆, 2₁1₄)     => -α^4//(α^5 + 15*α^4 + 85*α^3 + 225*α^2 + 274*α + 120)
  (3₁2₁1₁, 4₁1₂) => (-3*α^2 + 3*α)//(α^6 + 43//6*α^5 + 241//12*α^4 + 223//8*α^3…
  (3₁1₃, 5₁1₁)   => (5//6*α - 5//6)//(α^5 + 7*α^4 + 19*α^3 + 25*α^2 + 16*α + 4)
  (3₁1₃, 3₂)     => α//(α^5 + 7*α^4 + 19*α^3 + 25*α^2 + 16*α + 4)
  (3₂, 4₁2₁)     => (-1//3*α^2 - 1//12*α - 1//12)//(α^7 + 19//3*α^6 + 65//4*α^5…
  (4₁1₂, 4₁1₂)   => (1//6*α^2 - 1//3*α + 1//6)//(α^5 + 4*α^4 + 25//4*α^3 + 19//…
  (2₁1₄, 3₁1₃)   => (-3//2*α^3 + 15//2*α^2)//(α^5 + 13*α^4 + 63*α^3 + 143*α^2 +…
  (5₁1₁, 3₁1₃)   => (1//8*α - 1//40)//(α^5 + 38//15*α^4 + 143//60*α^3 + 21//20*…
  (1₆, 5₁1₁)     => α//(α^5 + 15*α^4 + 85*α^3 + 225*α^2 + 274*α + 120)
  (2₂1₂, 4₁1₂)   => (-1//2*α^3 + 17//2*α^2 - 3*α)//(α^6 + 23//2*α^5 + 51*α^4 + …
  (3₁1₃, 6₁)     => -1//(α^5 + 7*α^

Let's check that this matrix coincides with the $(\theta_{\lambda, \mu})^{-T}$

In [13]:
θ = generate_θ_matrix(N, α)
partitions = map(Partition, Combinatorics.integer_partitions(N))

lhs = MatrixSpace(S, length(partitions), length(partitions))()
for (n, λ) in enumerate(partitions), (m, ρ) in enumerate(partitions)
    lhs[n, m] = θ[(λ, ρ)]
end

inv_lhs = inv(lhs)

inv_θ = Dict()
for (n, λ) in enumerate(partitions), (m, ρ) in enumerate(partitions)
    inv_θ[(λ, ρ)] = inv_lhs[n, m]
end

for λ ∈ partitions, μ ∈ partitions
    @assert ω[(λ, μ)] == inv_θ[(μ, λ)]
end
println("We are awesome!")

We are awesome!


## Checking the hypothesis with Weingarten function

Consider the Weingarten function

$$
\mathrm{Wg}^\beta = \sum_{\lambda \vdash n} \frac{\omega_\lambda^{(\alpha)}(\cdot)}{J_\lambda^{(\alpha)}(d)},
$$
where $\alpha = 2 / \beta$.

We want to check the hypothesis that if we define matrix moments of the generalized orthogonal ensemble $^\beta O$ as follows:

$$
\int_{^\beta O}d{^\beta O}\,\,O_{i_1j_1}O_{i_2j_2}\dots O_{i_{2n}j_{2n}} = \sum_{\mathfrak{m}, \mathfrak{n} \in M_{2n}} \mathrm{Wg}^\beta([\mathfrak{m}, \mathfrak{n}]) \delta_\mathfrak{m}(i) \cdot \delta_\mathfrak{m}(i)
$$

then this distribution satisfies the following relation:

$$
\int_{^\beta O}d{^\beta O}\,\,J_\lambda^{(\alpha)}(AOBO^T) = \frac{J_\lambda^{(\alpha)}(A)J_\lambda^{(\alpha)}(B)}{J_\lambda^{(\alpha)}(1^d)}
$$

Here, $d$ is the size of the matrices, $A$ and $B$ are symmetric. The coset type $[\mathfrak{m}, \mathfrak{n}]$ can be defined as follows: consider the graph $G$ with the edges $\{\mathfrak{m}(2k - 1), \mathfrak{m}(2k)\}$ and $\{\mathfrak{n}(2k - 1), \mathfrak{n}(2k)\}$. It consists of cycles of even lengths $2\lambda_1 \ge 2\lambda_2 \ge \dots$. Then the coset type $[\mathfrak{m}, \mathfrak{n}]$ is the partition $\lambda$. 

This formula works for $\beta = 1$. We check that it does not work for other $\beta$.

First, we implement the function `get_pairing_permutations(indices)` that for the vector `indices=(i₁, i₂, ..., i₂ₙ)` returns all the pair partitions $\mathfrak{m}$ such that $\delta_{\mathfrak{m}}(i) = 1$. This function returns the pair partitions as permutations that matches to element the second element from the pair. 

In [14]:
using Permutations


function get_pairing_permutations(indices)
    n = length(indices)
    pairings = Permutation[]
    
    function step(pairs, last_l)
        l = last_l + 1
        while l ≤ n && pairs[l] != 0
            l += 1
        end
        if l == n + 1
            push!(pairings, Permutation(copy(pairs)))
            return
        end
        for r = (l + 1):n
            if !(pairs[r] == 0 && indices[l] == indices[r])
                continue
            end
            pairs[l] = r
            pairs[r] = l
            step(pairs, l)
            pairs[l] = 0
            pairs[r] = 0
        end
    end
    
    pairs = fill(0, n)
    step(pairs, 0)

    return pairings
end

get_pairing_permutations (generic function with 1 method)

Example:

In [15]:
get_pairing_permutations([1, 1, 1, 1, 2, 2, 3, 3])

3-element Vector{Permutation}:
 (1,2)(3,4)(5,6)(7,8)
 (1,3)(2,4)(5,6)(7,8)
 (1,4)(2,3)(5,6)(7,8)

Let $A = diag(a_1, a_2, \dots, ...)$ and let $B = diag(b_1, b_2, \dots)$. The integral

$$
\int_{^\beta O}d{^\beta O}\,\,p_\mu(AOBO^T)
$$

is equal to

$$
\int_{^\beta O}d{^\beta O}\,\,p_\mu(AOBO^T) = \sum_{\lambda \vdash n}\int_{^\beta O}d{^\beta O}\,\,\omega_\lambda(\mu) \cdot J_\lambda^{(\alpha)}(AOBO^T) = \sum_{\lambda \vdash n}\omega_\lambda(\mu) \cdot \frac{J_\lambda^{(\alpha)}(a_1, \dots, a_d)J_\lambda^{(\alpha)}(b_1, \dots, b_d)}{J_\lambda^{(\alpha)}(1^d)}
$$

On the other hand,

$$
p_\mu(AOBO^T) = \sum_{i_1, \dots, i_n}\sum_{j_1, \dots, j_n}\prod_{k=1}^na_{i_k}O_{i_kj_k}b_{j_k}O_{i_{\pi_\mu(k)}j_k},
$$


For each couple of set of indices $i = (i_1, \dots, i_n)$ and $j = (j_1, \dots, j_n)$ let's find we coefficient of the monomial $a_{i_1}a_{i_2}\dots a_{i_n}b_{i_1}\dots b_{j_n}$ in the integral

$$
\int_{^\beta O}d{^\beta O}\,\,p_\mu(AOBO^T) = \sum_{i_1, \dots, i_n}\sum_{j_1, \dots, j_n}a_{i_1}a_{i_2}\dots a_{i_n}b_{i_1}\dots b_{j_n}\int_{^\beta O}d{^\beta O} O_{i_1j_1}\dots O_{i_nj_n} \cdot O_{i_{\pi(1)}j_1}\dots O_{i_{\pi(n)}j_n}
$$

using the Weingarten function.

The following function takes a couple of list of indices $i=(i_1, \dots, i_{2n})$ and $j=(j_1, \dots, j_{2n})$ and for each partition $\lambda$ it returns the number of pair partitions $\mathfrak{m}$ and $\mathfrak{n}$ such that $\delta_{\mathfrak{m}}(i) = \delta_{\mathfrak{n}}(j) = 1$ and the coset type $[\mathfrak{m}, \mathfrak{n}]$ is $\lambda$.

In [16]:
function get_pairing_cosets(i_indices, j_indices)
    n = div(length(i_indices), 2)
    partitions = map(Partition, Combinatorics.integer_partitions(n))

    cosets = Dict(
        λ => 0 for λ ∈ partitions
    )
    for σ in get_pairing_permutations(i_indices), τ in get_pairing_permutations(j_indices)
        coset = Partition(sort(length.(Permutations.cycles(σ * τ)))[end:(-2):begin])
        cosets[coset] += 1
    end
    
    return cosets
end

get_pairing_cosets (generic function with 1 method)

The function `get_lhs_monomial_coef(μ, i_indices, j_indices)` takes the partition $\mu$ and a couple of indices $i=(i_1, \dots, i_n)$, $j=(j_1, \dots, j_n)$ and computes the coefficient of the monomial $a_{i_1}a_{i_2}\dots a_{i_n}b_{i_1}\dots b_{j_n}$ in the integral

$$
\int_{^\beta O}d{^\beta O}\,\,p_\mu(AOBO^T) = \sum_{i_1, \dots, i_n}\sum_{j_1, \dots, j_n}a_{i_1}a_{i_2}\dots a_{i_n}b_{i_1}\dots b_{j_n}\int_{^\beta O}d{^\beta O} O_{i_1j_1}\dots O_{i_nj_n} \cdot O_{i_{\pi(1)}j_1}\dots O_{i_{\pi(n)}j_n}.
$$

This coefficient can be represented as $\sum_{\rho \vdash n} \mathrm{coset}(\mu) \cdot Wg^{\beta}(\mu)$, and the function `get_lhs_monomial_coef(μ, i_indices, j_indices)` returns the vector `coset` -- number of couple of pairings $\mathfrak{m}$, $\mathfrak{n}$ with the coset type $\rho$ that matches reshuffled indices $i$ and $j$ with themselves.

In [17]:
function permutations(indices)
    perm = map(Permutation, Combinatorics.permutations(1:length(indices)))
    indices_perm = [
        [indices[σ.data[i]] for i = 1:length(indices)]
        for σ ∈ perm
    ]
    sort!(indices_perm)
    result = Vector{Int}[]
    for i = 1:length(indices_perm)
        if i == 1 || indices_perm[i] != indices_perm[i - 1]
            push!(result, indices_perm[i])
        end
    end
    return result
end

permutations (generic function with 1 method)

In [18]:
function get_lhs_monomial_coef(μ, i_indices, j_indices)
    # construct the permutation π_λ with the cycle decomposition type λ
    n = μ.n
    π_μ = Int[]
    position = 0
    for p in μ
        for i = 1:p
            push!(π_μ, (i % p) + 1 + position)
        end
        position += p
    end
    π_μ = Permutation(π_μ)
    
    partitions = map(Partition, Combinatorics.integer_partitions(μ.n))
    cosets = Dict(
        ρ => 0 for ρ ∈ partitions
    )
    
    for iₐ in permutations(i_indices), jₐ in permutations(j_indices)
        iₐ_prime = [iₐ[π_μ[i]] for i = 1:n]
        jₐ_prime = copy(jₐ)
        current_cosets = get_pairing_cosets(
            vcat(iₐ, iₐ_prime),
            vcat(jₐ, jₐ_prime),
        )
        for ρ ∈ partitions
            cosets[ρ] += current_cosets[ρ]
        end
    end
    return cosets
end

get_lhs_monomial_coef (generic function with 1 method)

Example

In [19]:
i_indices = [1, 2, 3, 4, 5]
j_indices = [1, 2, 3, 4, 5]
μ = Partition([3, 2])

get_lhs_monomial_coef(μ, i_indices, j_indices)

Dict{AbstractAlgebra.Generic.Partition{Int64}, Int64} with 7 entries:
  5₁   => 0
  4₁1₁ => 0
  2₁1₃ => 0
  3₁1₂ => 0
  3₁2₁ => 14400
  2₂1₁ => 0
  1₅   => 0

Generally, one can prove that if $i = (1, 2, \dots, n)$, $j=(1, 2, \dots, n)$, then $\mathrm{coset}(\rho) = (n!)^2 \delta_{\mu\rho}$

The sum can be rewritten as

$$
\sum_{\rho \vdash n} \mathrm{coset}(\rho) \cdot Wg^{\beta}(\rho) = \sum_{\lambda \vdash n} \frac{\sum_{\rho \vdash n}\mathrm{coset}(\rho) \cdot \omega_\lambda(\rho)}{J_\lambda^{(\alpha)}(1^d)}
$$

So, for each $\lambda$ let's evaluate $\sum_{\rho \vdash n}\mathrm{coset}(\rho) \cdot \omega_\lambda(\rho)$

In [20]:
function get_lhs_monomial_coef_2(μ, i_indices, j_indices, ω)
    partitions = map(Partition, Combinatorics.integer_partitions(μ.n))
    cosets = get_lhs_monomial_coef(μ, i_indices, j_indices)
    
    coefs = Dict(
        λ => sum(cosets[ρ] * ω[(λ, ρ)] for ρ ∈ partitions)
        for λ ∈ partitions 
    )
    return coefs
end


get_lhs_monomial_coef_2 (generic function with 1 method)

Example

In [21]:
i_indices = [1, 2, 3, 4, 5]
j_indices = [1, 2, 3, 4, 5]
μ = Partition([3, 2])

ω = generate_ω_matrix(5, α)

get_lhs_monomial_coef_2(μ, i_indices, j_indices, ω)

Dict{AbstractAlgebra.Generic.Partition{Int64}, AbstractAlgebra.Generic.Frac{AbstractAlgebra.Generic.Poly{Rational{BigInt}}}} with 7 entries:
  5₁   => 600//(α^4 + 25//12*α^3 + 35//24*α^2 + 5//12*α + 1//24)
  4₁1₁ => -3000//(α^4 + 29//12*α^3 + 49//24*α^2 + 17//24*α + 1//12)
  2₁1₃ => (36000*α)//(α^4 + 17//2*α^3 + 49//2*α^2 + 29*α + 12)
  3₁1₂ => (-7200*α + 7200)//(α^4 + 25//6*α^3 + 19//3*α^2 + 25//6*α + 1)
  3₁2₁ => (7200*α^2 + 7200)//(α^5 + 29//6*α^4 + 17//2*α^3 + 41//6*α^2 + 5//2*α …
  2₂1₁ => (-21600*α^2 - 21600)//(α^5 + 15//2*α^4 + 41//2*α^3 + 51//2*α^2 + 29//…
  1₅   => (-14400*α)//(α^4 + 10*α^3 + 35*α^2 + 50*α + 24)

Finally, we need to compute the coefficient of $a_{i_1}\dots a_{i_n}b_{j_1}\dots b_{j_n}$ in the right-hand side of the equation:

$$
\int_{^\beta O}d{^\beta O}\,\,p_\mu(AOBO^T) = \sum_{\lambda \vdash n}\int_{^\beta O}d{^\beta O}\,\,\omega_\lambda(\mu) \cdot J_\lambda^{(\alpha)}(AOBO^T) = \sum_{\lambda \vdash n}\omega_\lambda(\mu) \cdot \frac{J_\lambda^{(\alpha)}(a_1, \dots, a_d)J_\lambda^{(\alpha)}(b_1, \dots, b_d)}{J_\lambda^{(\alpha)}(1^d)}
$$

For every integer $k$, let $t_k$ denotes the number of $k$ in the set of indices $i$. Then denote $\lambda(i) = (t_1, t_2, \dots, ) \vdash n$. Let $T^{jp}$ be a translation matrix from $J_\lambda$ to the monomial basis:

$$
J_\lambda = \sum_{\mu \vdash n} T^{jp}(\lambda, \mu) m_\mu
$$

Then the coefficient of $a_{i_1}\dots a_{i_n}b_{j_1}\dots b_{j_n}$ in the right-hand side of the equation using the following formula:

$$
[a_{i_1}\dots a_{i_n}b_{j_1}\dots b_{j_n}] = \sum_{\lambda \vdash n} \frac{\omega_\lambda(\mu) \cdot T^{jm}(\lambda, \lambda(i)) \cdot T^{jm}(\lambda, \lambda(j))}{J_\lambda^{(\alpha)}(1^d)}
$$

If the coefficients in both left- and right-hand sides are identical, then we may conclude that the equality

$$
\sum_{\rho \vdash n}\mathrm{coset}(\rho) \cdot \omega_\lambda(\rho) = \omega_\lambda(\mu) \cdot T^{jm}(\lambda, \lambda(i)) \cdot T^{jm}(\lambda, \lambda(j))
$$

holds for all $\lambda$ (because the denominator is a polynomial that depends on the additional variable $d$; this is not a rigorous proof)

To compare the coefficients, we need to compute the translation matrix $T^{jp}$. I used the paper
   `Merca, Mircea. "Augmented monomials in terms of power sums." SpringerPlus 4.1 (2015): 1-13.` to evaluate the matrix $T^{mp}$ from the monomial basis to the power sum one. After that, I inverted it (the resulting matrix is the translation matrix for power sum basis to the monomial one), and multiplied it by $\theta$ from the left. 

First, we implement the majorization sort on the set of partitions of the size $n$. The function `majorization_sort` returns the vector of partitions sorted in descending order.

In [22]:
function ⪰(λ::AbstractPartition, μ::AbstractPartition)
    partial_sum = 0
    for k = 1:min(length(λ), length(μ))
        partial_sum += λ.part[k] - μ.part[k]
        if partial_sum < 0
            return false
        end
    end
    return true
end

function majorization_sort(partitions::Vector{AbstractPartition})
    edges = Vector{Int}[Int[] for i = 1:length(partitions)]
    for (n, λ) ∈ enumerate(partitions), (m, μ) ∈ enumerate(partitions)
        if ⪰(λ, μ)
            push!(edges[n], m)
        end
    end
    
    sorted_partitions = AbstractPartition[]
    visited = [false for i = 1:length(partitions)]
    
    function dfs(n)
        if visited[n]
            return
        end
        visited[n] = true
        for m in edges[n]
            dfs(m)
        end
        push!(sorted_partitions, partitions[n])
    end
    
    for i = 1:length(partitions)
        if visited[i]
            continue
        end
        dfs(i)
    end
    
    return sorted_partitions[end:-1:begin]
end

majorization_sort (generic function with 1 method)

Next, we write the function that computes the matrix $T^{mp}$

In [23]:
function Tₘₚ(n::Int)
    T = Dict()
    T[(Partition([1]), Partition([1]))] = 1
    for k = 2:n
        partitions = majorization_sort(
            map(Partition, Combinatorics.integer_partitions(k))
        )
        λ = Partition([k])
        T[(λ, λ)] = 1
        for μ ∈ partitions[2:end]
            T[(λ, μ)] = 0
        end
        for (p, λ) ∈ enumerate(partitions[2:end])
            p += 1
            for μ ∈ partitions[1:(p - 1)]
                v = zeros(Int64, μ.part[1])
                for i = 1:length(μ)
                    v[μ.part[i]] += 1
                end

                t = zeros(Int64, 2 * λ.part[1] + 1)
                for i = 1:length(λ)
                    t[λ.part[i]] += 1
                end

                j = λ.part[1]
                if j ≤ length(v) && v[j] > 0
                    μ₀ = Int[]
                    for r = 1:length(v)
                        v₀ = (r == j ? v[r] - 1 : v[r])
                        for ~ = 1:v₀
                            push!(μ₀, r)
                        end
                    end
                    μ₀ = Partition(μ₀[end:-1:begin])
                    
                    λ₀ = Int[]
                    for r = 1:length(t)
                        t₀ = (r == j ? t[r] - 1 : t[r])
                        for ~ = 1:t₀
                            push!(λ₀, r)
                        end
                    end
                    λ₀ = Partition(λ₀[end:-1:begin])

                    T[(λ, μ)] = T[(λ₀, μ₀)]
                else
                    T[(λ, μ)] = 0
                end

                for i = 1:length(t)
                    δᵢⱼ = (i == j ? 1 : 0)
                    if t[i] ≤ δᵢⱼ
                        continue
                    end
                    λᵢ = Int[]
                    for r = 1:length(t)
                        tᵢ = t[r]
                        if r == i + j
                            tᵢ += 1
                        elseif r == i || r == j
                            tᵢ -= 1 + δᵢⱼ
                        end
                        for ~ = 1:tᵢ
                            push!(λᵢ, r)
                        end
                    end
                    λᵢ = Partition(λᵢ[end:-1:begin])
                    T[(λ, μ)] -= (t[i] - δᵢⱼ) * T[(λᵢ, μ)]
                end
            end
        
            T[(λ, λ)] = 1

            for μ ∈ partitions[(p + 1):end]
                T[(λ, μ)] = 0
            end
        end
    end
    
    partitions = majorization_sort(
        map(Partition, Combinatorics.integer_partitions(n))
    )
        
    for λ ∈ partitions, μ ∈ partitions 
        t = zeros(Int64, λ.part[1])
        for i = 1:length(λ)
            t[λ.part[i]] += 1
        end
        for i = 1:length(t)
            T[(λ, μ)] //= factorial(t[i])
        end
    end
    
    return T
end


function Tₚₘ(n::Int)
    T = Tₘₚ(n)
    partitions = map(Partition, Combinatorics.integer_partitions(n))
    
    matrix = zeros(Rational{BigInt}, length(partitions), length(partitions))
    for (i, λ) ∈ enumerate(partitions), (j, μ) ∈ enumerate(partitions)
        matrix[i, j] = T[(λ, μ)]
    end
    inv_matrix = inv(matrix)
    
    result = Dict()
    for (i, λ) ∈ enumerate(partitions), (j, μ) ∈ enumerate(partitions)
        result[(λ, μ)] = inv_matrix[i, j]
    end
    return result
end

Tₚₘ (generic function with 1 method)

Example:

In [24]:
Tₚₘ(4)

Dict{Any, Any} with 25 entries:
  (3₁1₁, 4₁)   => 1//1
  (2₂, 3₁1₁)   => 0//1
  (2₂, 2₂)     => 2//1
  (4₁, 2₁1₂)   => 0//1
  (4₁, 1₄)     => 0//1
  (1₄, 3₁1₁)   => 4//1
  (1₄, 2₂)     => 6//1
  (2₂, 4₁)     => 1//1
  (1₄, 4₁)     => 1//1
  (4₁, 3₁1₁)   => 0//1
  (4₁, 2₂)     => 0//1
  (2₁1₂, 3₁1₁) => 2//1
  (4₁, 4₁)     => 1//1
  (2₁1₂, 1₄)   => 0//1
  (2₂, 2₁1₂)   => 0//1
  (3₁1₁, 3₁1₁) => 1//1
  (3₁1₁, 1₄)   => 0//1
  (2₁1₂, 2₂)   => 2//1
  (2₂, 1₄)     => 0//1
  (1₄, 2₁1₂)   => 12//1
  (3₁1₁, 2₂)   => 0//1
  (1₄, 1₄)     => 24//1
  (2₁1₂, 2₁1₂) => 2//1
  (3₁1₁, 2₁1₂) => 0//1
  (2₁1₂, 4₁)   => 1//1

Now, we can compute the matrix $T^{jm}$:

In [25]:
function Tⱼₘ(n::Int, θ)
    partitions = map(Partition, Combinatorics.integer_partitions(n))
    T = Dict(
        (λ, μ) => zero(α)
        for λ ∈ partitions, μ ∈ partitions 
    )
    
    T_pm = Tₚₘ(n)
    for λ ∈ partitions, μ ∈ partitions, ρ ∈ partitions
        T[(λ, μ)] += θ[(λ, ρ)] * T_pm[(ρ, μ)]
    end
    return T
end

Tⱼₘ (generic function with 1 method)

**Example:**

In [26]:
Tⱼₘ(5, θ)

Dict{Tuple{AbstractAlgebra.Generic.Partition{Int64}, AbstractAlgebra.Generic.Partition{Int64}}, AbstractAlgebra.Generic.Poly{Rational{BigInt}}} with 49 entries:
  (2₁1₃, 2₂1₁) => 0
  (2₁1₃, 3₁2₁) => 0
  (2₂1₁, 1₅)   => 120
  (1₅, 2₂1₁)   => 0
  (4₁1₁, 2₁1₃) => 36*α + 54
  (3₁1₂, 5₁)   => 0
  (3₁2₁, 1₅)   => 120
  (4₁1₁, 3₁1₂) => 16*α^2 + 30*α + 14
  (5₁, 2₂1₁)   => 30*α^2 + 60*α + 30
  (3₁1₂, 4₁1₁) => 0
  (1₅, 2₁1₃)   => 0
  (2₁1₃, 2₁1₃) => 6*α + 24
  (2₁1₃, 3₁1₂) => 0
  (5₁, 2₁1₃)   => 60*α + 60
  (4₁1₁, 5₁)   => 0
  (1₅, 3₁2₁)   => 0
  (4₁1₁, 4₁1₁) => 6*α^3 + 13*α^2 + 9*α + 2
  (5₁, 3₁2₁)   => 20*α^3 + 50*α^2 + 40*α + 10
  (2₂1₁, 2₂1₁) => 2*α^2 + 10*α + 12
  (2₂1₁, 3₁2₁) => 0
  (3₁2₁, 2₂1₁) => 6*α^2 + 22*α + 20
  (3₁2₁, 3₁2₁) => 2*α^3 + 8*α^2 + 10*α + 4
  (2₁1₃, 5₁)   => 0
  (3₁1₂, 1₅)   => 120
  (2₁1₃, 4₁1₁) => 0
  ⋮            => ⋮

 This example shows that the coefficient of the monomial symmetric function $m_{1^n}$ is equal to $n!$ for all $\lambda$, which coincides with the theoretical value. In particular, this means that the formula
 
 $$
 \sum_{\rho \vdash n}\mathrm{coset}(\rho) \cdot \omega_\lambda(\rho) = \omega_\lambda(\mu) \cdot T^{jm}(\lambda, \lambda(i)) \cdot T^{jm}(\lambda, \lambda(j))
 $$
 
 automatically holds if $i=(1, \dots, n)$ and $j=(1, \dots, n)$: both left- and right-hand sides are equal to $(n!)^2 \cdot \omega_\lambda(\mu)$.

Now, it's time to check the equation:

In [27]:
permutations(i_indices)

120-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5]
 [1, 2, 3, 5, 4]
 [1, 2, 4, 3, 5]
 [1, 2, 4, 5, 3]
 [1, 2, 5, 3, 4]
 [1, 2, 5, 4, 3]
 [1, 3, 2, 4, 5]
 [1, 3, 2, 5, 4]
 [1, 3, 4, 2, 5]
 [1, 3, 4, 5, 2]
 [1, 3, 5, 2, 4]
 [1, 3, 5, 4, 2]
 [1, 4, 2, 3, 5]
 ⋮
 [5, 3, 1, 2, 4]
 [5, 3, 1, 4, 2]
 [5, 3, 2, 1, 4]
 [5, 3, 2, 4, 1]
 [5, 3, 4, 1, 2]
 [5, 3, 4, 2, 1]
 [5, 4, 1, 2, 3]
 [5, 4, 1, 3, 2]
 [5, 4, 2, 1, 3]
 [5, 4, 2, 3, 1]
 [5, 4, 3, 1, 2]
 [5, 4, 3, 2, 1]

In [28]:
n = 4
ω = generate_ω_matrix(n, α)
T_jm = Tⱼₘ(n, θ)

i_indices = [1, 1, 1, 2]
λᵢ = Partition([3, 1])

j_indices = [1, 1, 2, 2]
λⱼ = Partition([2, 2])

μ = Partition([3, 1])

println(get_lhs_monomial_coef(μ, i_indices, j_indices))

lhs = get_lhs_monomial_coef_2(μ, i_indices, j_indices, ω)
partitions = map(Partition, Combinatorics.integer_partitions(n))

rhs = Dict(
    λ => ω[(λ, μ)] * T_jm[(λ, λᵢ)] * T_jm[(λ, λⱼ)]
    for λ in partitions
)
diff = Dict(
    λ => lhs[λ] - rhs[λ]
    for λ ∈ partitions 
)

Dict{AbstractAlgebra.Generic.Partition{Int64}, Int64}(3₁1₁ => 1008, 1₄ => 36, 4₁ => 1440, 2₂ => 360, 2₁1₂ => 396)


Dict{AbstractAlgebra.Generic.Partition{Int64}, AbstractAlgebra.Generic.Frac{AbstractAlgebra.Generic.Poly{Rational{BigInt}}}} with 5 entries:
  3₁1₁ => (-4*α^4 - 8*α^3 + 980*α - 1832)//(α^3 + 7//3*α^2 + 5//3*α + 1//3)
  1₄   => (36*α^3 - 396*α^2 + 1368*α - 1440)//(α^3 + 6*α^2 + 11*α + 6)
  4₁   => (-8*α^4 - 28*α^3 - 36*α^2 - 20*α + 536)//(α^3 + 11//6*α^2 + α + 1//6)
  2₂   => (864*α^2 - 2268*α + 1080)//(α^4 + 9//2*α^3 + 7*α^2 + 9//2*α + 1)
  2₁1₂ => (612*α^2 - 3420*α + 4392)//(α^3 + 5*α^2 + 7*α + 3)

As we can see, all the entries are non-zero. At the same time, they are zero for $\alpha=2 \Leftrightarrow \beta=1$.

In [29]:
for λ ∈ partitions
    println("$(λ) => $(evaluate(diff[λ], 2//1))")
end

1₄ => 0//1
2₁1₂ => 0//1
2₂ => 0//1
3₁1₁ => 0//1
4₁ => 0//1


So, probably we don't have any obvious mistakes.